In [2]:
import os
from pathlib import Path
home = os.getcwd()
current = home
while 'data' not in os.listdir(current):
    current = Path(current).parent
DATA_FOLDER = os.path.join(current, 'data')

In [3]:
amazon_dir = os.path.join(DATA_FOLDER, 'amazon', 'amazon')
amazon_train = os.path.join(DATA_FOLDER, 'amazon', 'amazon_splitted', 'train')
amazon_val = os.path.join(DATA_FOLDER, 'amazon', 'amazon_splitted', 'val')